In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns 
from sklearn import preprocessing
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory
from subprocess import check_output
print(check_output(["ls", ".."]).decode("utf8"))
# Any results you write to the current directory are saved as output.

In [ ]:
train = pd.read_table("../input/train.tsv")
test = pd.read_table("../input/test_stg2.tsv")

In [ ]:
print(train.shape)
print(test.shape)

**Dependent/Target  Variable  Analysis: Price**

In [ ]:
"""train['price'].describe()"""

In [ ]:
"""plt.subplot(1,2,1)
train['price'].plot.hist(bins=60, figsize=(20,10), edgecolor="black",range=[0,290])
plt.xlabel("Price range" , fontsize=20)
plt.ylabel("Frequency", fontsize=20)
plt.tick_params(labelsize = 15)
plt.title(" Distribution of Price", fontsize=20)

# Tranforming price using log as price appears to have a left skew 

plt.subplot(1,2,2)
np.log(train['price']+1).plot.hist(bins=60,edgecolor="black",figsize=(20,10))
plt.xlabel("Price Range with Log transformation (log(price + 1))", fontsize=20)
plt.ylabel("Frequency", fontsize=20)
plt.tick_params(labelsize = 15)
plt.title(" Distribution of Log Transformed Price", fontsize=20)"""

**Shipping Variable**

In [ ]:
"""# So over half of the values shipping is not free
train['shipping'].astype('category').value_counts()/len(train)"""

Clearly seen from the histograms below that price is a little higher when product does have shipping included 

In [ ]:
"""plt.subplot(1,2,1)
np.log(train.loc[train['shipping']==1,'price']+1).plot.hist(bins=20, figsize=(20,10), edgecolor="black")
plt.xlabel("Price range (Shipping is free)" , fontsize=20)
plt.ylabel("Frequency", fontsize=20)
plt.tick_params(labelsize = 15)
plt.title("Price Distribution Shipping Free", fontsize=20)

plt.subplot(1,2,2)
np.log(train.loc[train['shipping']==0,'price']+1).plot.hist(bins=20,edgecolor="black",figsize=(20,10))
plt.xlabel("Price Range (Shipping payed by user)", fontsize=20)
plt.ylabel("Frequency", fontsize=20)
plt.tick_params(labelsize = 15)
plt.title("Price Distribution Shipping Charged", fontsize=20)"""

**Item Condition **

In [ ]:
"""# Lets see how price distributes over various item_conditions 
train['item_condition_id'].value_counts()/len(train)"""

*Looking at the plots it looks like item condition doesnt asy anything about the price of an item on its own *

In [ ]:
"""plt.figure(figsize=(10,5))
ax = sns.boxplot(x=np.log(train['price']+1),y='item_condition_id',
                 data=train, dodge=False ,  orient="h", palette="Set1")
plt.title("Price Distribution with Item_condition",fontsize=15)
plt.xlabel(" Log transformed price range",fontsize = 10)
plt.ylabel("Item quality with 1 (Best) - 5(Worst)",fontsize = 10)
plt.tick_params(labelsize=15)"""

**Category**

In [ ]:
"""all_categories = pd.Series([cat for cat_list in list(train['category_name'].apply(lambda x:str(x).split("/"))) for cat in cat_list])
print("There are a total of {0} categories of products".format(all_categories.value_counts().shape[0]))
print("Top 10 categories with occurence frequency:")
all_categories.value_counts()[:10]/len(all_categories)*100"""

In [ ]:
"""print("There are {0} products without any categories".format(train['category_name'].isnull().sum()))"""

In [ ]:
###
# reference: https://www.kaggle.com/thykhuely/mercari-interactive-eda-topic-modelling/notebook
import re
def split_cat(text):
    try: 
        return text.split("/")
    except: return ("Unknow", "Unknown", "Unknown")

from sklearn.preprocessing import LabelEncoder as le
def encode_labels(df, columns):
    res = []
    le = preprocessing.LabelEncoder()
    for col in columns:
        print("Encoding %s"%col)
        res.append(le.fit_transform(df[col].tolist()))
    return tuple(res)

train['general_cat'], train['subcat_1'], train['subcat_2'] = \
zip(*train['category_name'].apply(lambda x: split_cat(x)))
train.head()



In [ ]:
###
#Same operations on test 
test['general_cat'], test['subcat_1'], test['subcat_2'] = \
zip(*test['category_name'].apply(lambda x: split_cat(x)))
test.head()


Clearly categories define some price differences and we need to vectorize these variables to use them 

In [ ]:
"""#Price distribution of top 10 caqtegories 
top_cat = train.loc[train['general_cat'].isin(list(all_categories.value_counts().index[:10])),['general_cat','price']]
plt.figure(figsize=(10,5))
ax = sns.boxplot(x=np.log(top_cat['price']+1),y='general_cat',
                 data=top_cat, dodge=False ,  orient="h", palette="Set2")
plt.title("Price Distribution with Top Categories",fontsize=15)
plt.xlabel("Log transformed price range",fontsize = 10)
plt.ylabel("Item category",fontsize = 10)
plt.tick_params(labelsize=15)"""

In [ ]:
###
# fILL NULL BRNADS WITH OTHER AND LABEL ENCODE ALL BRANDS 
train['brand_name'].fillna("Other",inplace = True)
columns = ['brand_name','general_cat','subcat_1','subcat_2']
train['brand_name'], train['general_cat'],train['subcat_1'],train['subcat_2'] = encode_labels(train,columns)
test['brand_name'], test['general_cat'],test['subcat_1'],test['subcat_2'] = encode_labels(test,columns)

In [ ]:
###
# reference: https://ahmedbesbes.com/how-to-mine-newsfeed-data-and-extract-interactive-insights-in-python.html
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize
import re
import string 
stop = set(stopwords.words('english'))
def tokenize(text):
    """
    sent_tokenize(): segment text into sentences
    word_tokenize(): break sentences into words
    """
    try: 
        regex = re.compile('[' +re.escape(string.punctuation) + '0-9\\r\\t\\n]')
        text = regex.sub(" ", text) # remove punctuation
        
        tokens_ = [word_tokenize(s) for s in sent_tokenize(text)]
        tokens = []
        for token_by_sent in tokens_:
            tokens += token_by_sent
        tokens = list(filter(lambda t: t.lower() not in stop, tokens))
        filtered_tokens = [w for w in tokens if re.search('[a-zA-Z]', w)]
        filtered_tokens = [w.lower() for w in filtered_tokens if len(w)>=3]
        
        return filtered_tokens
            
    except TypeError as e: print(text,e)

In [ ]:
###
# Drop null item_descriptions as tfidf does not handle nans
print("{0} missing values in train['item_description']".format(test['item_description'].isnull().sum()))
print("{0} missing values in test['item_description']".format(train['item_description'].isnull().sum()))
train.fillna("No description yet",inplace =True)
test.fillna("No description yet",inplace =True)
print("{0} missing values in train['item_description']".format(train['item_description'].isnull().sum()))
print("{0} missing values in test['item_description']".format(test['item_description'].isnull().sum()))

In [ ]:
###
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(min_df=10,max_features =100000,tokenizer=tokenize, ngram_range=(1, 2))
import time
start = time.time()
all_tfidf = vectorizer.fit_transform(train['item_description'].tolist() + test['item_description'].tolist())
print("Vectorizing the input dataframe this make take some time...")
train_tfidf = vectorizer.transform(train['item_description'].tolist())
test_tfidf = vectorizer.transform(test['item_description'].tolist())
print("Time taken for Vectorizer {0} seconds".format(time.time() - start))

In [ ]:
###
from sklearn.decomposition import TruncatedSVD
n_comp=30
svd_obj = TruncatedSVD(n_components=n_comp, random_state=42)
start = time.time()
print("Now Generating {0} components using Singular value Decomposition...".format(n_comp))
svd_obj.fit(all_tfidf)
train_svd = pd.DataFrame(svd_obj.transform(train_tfidf))
test_svd = pd.DataFrame(svd_obj.transform(test_tfidf))
    
train_svd.columns = ['svd_item_description_'+str(i) for i in range(n_comp)]
test_svd.columns = ['svd_item_description_'+str(i) for i in range(n_comp)]
train= pd.concat([train, train_svd], axis=1)
test = pd.concat([test, test_svd], axis=1)
print("Time taken for SVD {0} seconds".format(time.time() - start))

In [ ]:
###
start = time.time()
vectorizer = TfidfVectorizer(min_df=10,max_features =100000,tokenizer=tokenize, ngram_range=(1, 2))
all_tfidf = vectorizer.fit_transform(train['name'].tolist() + test['name'].tolist())
print("Vectorizing the input dataframe this make take some time...")
train_tfidf = vectorizer.transform(train['name'].tolist())
test_tfidf = vectorizer.transform(test['name'].tolist())
print("Time taken for Vectorizer {0} seconds".format(time.time() - start))

start = time.time()
n_comp=30
svd_obj = TruncatedSVD(n_components=n_comp, random_state=42)
print("Now Generating {0} components using Singular value Decomposition...".format(n_comp))
svd_obj.fit(all_tfidf)
train_svd = pd.DataFrame(svd_obj.transform(train_tfidf))
test_svd = pd.DataFrame(svd_obj.transform(test_tfidf))
    
train_svd.columns = ['svd_name'+str(i) for i in range(n_comp)]
test_svd.columns = ['svd_name'+str(i) for i in range(n_comp)]
train_df = pd.concat([train, train_svd], axis=1)
test_df = pd.concat([test, test_svd], axis=1)
print("Time taken for SVD {0} seconds".format(time.time() - start))

In [ ]:
###
train_df = train.copy()
test_df = test.copy()
print("Difference of features in train and test are {}".format(np.setdiff1d(train.columns, test.columns)))
print("")
do_not_use_for_training = ['cat_1','test_id','cat_2','cat_3','train_id','name', 'category_name', 'brand_name', 'price', 'item_description']
feature_names = [f for f in train.columns if f not in do_not_use_for_training]
print("We will be using following features for training {}.".format(feature_names))
print("")
print("Total number of features are {}.".format(len(feature_names)))

In [ ]:
###
y = np.log(train['price'].values + 1)

In [ ]:
###
import xgboost as xgb
from sklearn.model_selection import train_test_split
Xtr, Xv, ytr, yv = train_test_split(train[feature_names].values, y, test_size=0.2, random_state=1987)
dtrain = xgb.DMatrix(Xtr, label=ytr)
dvalid = xgb.DMatrix(Xv, label=yv)
dtest = xgb.DMatrix(test[feature_names].values)
watchlist = [(dtrain, 'train'), (dvalid, 'valid')]

start = time.time()
xgb_par = {'min_child_weight': 20, 'eta': 0.05, 'colsample_bytree': 0.5, 'max_depth': 15,
            'subsample': 0.9, 'lambda': 2.0, 'nthread': -1, 'booster' : 'gbtree', 'silent': 1,
            'eval_metric': 'rmse', 'objective': 'reg:linear'}

model_1 = xgb.train(xgb_par, dtrain, 80, watchlist, early_stopping_rounds=20, maximize=False, verbose_eval=20)
print('Modeling RMSLE %.5f' % model_1.best_score)
end = time.time()
print("Time taken in training is {}.".format(end - start))

In [ ]:
###
start = time.time()
yvalid = model_1.predict(dvalid)
ytest = model_1.predict(dtest)
end = time.time()
print("Time taken in prediction is {}.".format(end - start))

In [ ]:
###
start = time.time()
if test.shape[0] == ytest.shape[0]:
    print('Test shape OK.') 
test['price'] = np.exp(ytest) - 1
test[['test_id', 'price']].to_csv('xgb_submission_mercari.csv', index=False)
end = time.time()
print("Time taken in training is {}.".format(end - start))